In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

pd.options.display.max_columns = None

In [2]:
datos = ['https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=60',
        'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=120', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=180',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=240', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=300',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=360', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=420']

In [3]:
preciojug = pd.DataFrame()

for i in datos:
    url = i
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    html.status_code
    
    productos = soup.find_all("td",{"class": "col col-vl"})
    
    valor = {'precio': []}
    
    for i in productos:
        valor['precio'].append(i) 
        
    precio = pd.DataFrame(valor)
    
    precio["precio"] = precio['precio'].astype(str)
    
    precio['precio'] = precio['precio'].str.extract(r'(?<=\"\>)(.*)(?=\<\/td\>)')
    
    
    
    preciojug = pd.concat([preciojug, precio], axis=0)

In [4]:
preciojug = preciojug.reset_index(drop=True)
preciojug.head(2)

,precio
0,€3.3M
1,€3.8M


In [5]:
preciojug = preciojug["precio"].replace({'€': '', "K":"*1e3", "M":"*1e6"}, regex=True).map(pd.eval).astype(int)
preciojug = pd.DataFrame(preciojug)

In [6]:
preciojug.head()

,precio
0,3300000
1,3800000
2,3600000
3,3500000
4,3600000


In [7]:
preciojug = preciojug.drop(preciojug.index[[31,80,89,95,116,158,208,263]])
preciojug = preciojug.reset_index(drop=True)
preciojug.head(2)

,precio
0,3300000
1,3800000


In [8]:
preciojug.head(2)

,precio
0,3300000
1,3800000


In [10]:
preciojug.to_csv('../../data/precio/precios_jugadores.csv')